In [1]:
import sys
import os
from pathlib import Path

# Caminho absoluto para a raiz do projeto (um nível acima de notebooks)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)


In [2]:
from config.utils import *


In [6]:
path = os.path.join(PATHS['SILVER_LAYER'],"cny_brl.parquet")

In [7]:
df = pd.read_parquet(path)

In [8]:
path_cny = os.path.join(PATHS['METRICS'],'schemas','cny_brl.yaml')

In [11]:
SCHEMAS['ipca']

<Schema DataFrameSchema(columns={'data': <Schema Column(name=data, type=DataType(datetime64[ns]))>, 'ipca': <Schema Column(name=ipca, type=DataType(float64))>}, checks=[], parsers=[], index=None, dtype=None, coerce=False, strict=False, name=IPCA, ordered=False, unique=None, report_duplicates=all, unique_column_names=False, add_missing_columns=False, title=None, description=Índice de Preços ao Consumidor Amplo (IBGE). Série histórica mensal., metadata=None, drop_invalid_rows=False)>

In [ ]:
#Ingestão dos dados na camada silver
# pyrefly: ignore  # unknown-name
for nome_indicador, parametros in INDICADORES.items():
    fonte = parametros.get('FONTE')

    # pyrefly: ignore  # unknown-name
    dados = ler_dados_indicadores(nome_indicador=nome_indicador, nome_fonte=fonte)

    if dados is not None:
        # pyrefly: ignore  # unknown-name
        salvar_dados_indicadores(dados=dados, nome_indicador=nome_indicador)
    else:
        # pyrefly: ignore  # unknown-name
        logger.warning(f"Dados do indicador '{nome_indicador}' não foram carregados.")


In [5]:
def ler_dados_indicadores_silver(nome_indicador: str, path: str = PATHS['SILVER_LAYER']) -> pd.DataFrame | None:

    """
    Lê um arquivo Parquet da camada silver e retorna um DataFrame ou None.

    Args:
        nome_indicador (str): Nome do indicador (ex: 'IPCA').
        path (str): Caminho da pasta onde está o Parquet.

    Returns:
        pd.DataFrame: DataFrame formatado ou None se houver erro.
    """
    
    try:
        
        nome_indicador = nome_indicador.lower()
        nome_arquivo: str = f"{nome_indicador}.parquet"
        path_arquivo = os.path.join(path, nome_arquivo)

        logger.info(f"Lendo arquivo: {path_arquivo}")

        df = pd.read_parquet(path_arquivo)
       
        return df
    except FileNotFoundError:
        logger.warning(f"Arquivo não encontrado: {nome_arquivo}")
    except Exception as e:
        logger.error(f"Erro ao ler {nome_indicador}: {e}", exc_info=True)
    return None

In [16]:
indicador = ler_dados_indicadores_silver('cny_brl')

2025-07-27 02:16:14.407 | INFO     | __main__:ler_dados_indicadores_silver:20 - Lendo arquivo: C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data/silver\cny_brl.parquet


In [12]:
import pandera.pandas as pa

In [15]:
schema_path = os.path.join(PATHS['SCHEMAS'],f"{nome_indicador}.yaml")
pa.DataFrameSchema.from_yaml(schema_path).validate(df_indicador, lazy=True)

NameError: name 'nome_indicador' is not defined

In [22]:
def validar_schema(df_indicador: pd.DataFrame, nome_indicador: str) -> bool:
    """
    Valida um DataFrame usando um schema Pandera carregado de um arquivo YAML.

    Args:
        df (pd.DataFrame): DataFrame a ser validado.
        nome_indicador (str): Nome do schema (ex: 'ipca', 'usd_brl').
    Returns:
        bool: True se a validação for bem-sucedida, False caso mal-sucedida.
    """
    try:
        schema_path = os.path.join(PATHS['SCHEMAS'],f"{nome_indicador}.yaml")
        pa.DataFrameSchema.from_yaml(schema_path).validate(df_indicador, lazy=True)
        logger.success(f"Schema '{nome_indicador}' validado!")
        return True
    except pa.errors.SchemaErrors as e:
        logger.error(f"Erros de validação no schema '{nome_indicador}':\n\n{e.failure_cases}")
    except Exception as e:
        logger.error(f"Erro ao validar '{nome_indicador}': {e}")
    return False

In [23]:
validar_schema(indicador,'ipca')

2025-07-27 02:18:57.117 | ERROR    | __main__:validar_schema:19 - Erro ao validar 'ipca': Schema representation must be a mapping.


False

In [ ]:
def salvar_dados_indicadores_gold(df: pd.DataFrame, nome_indicador: str) -> None:
    """
    Salva um DataFrame na camada Gold no formato Parquet.

    Args:
        df (pd.DataFrame): DataFrame a ser salvo.
        nome_indicador (str): Nome do indicador (chave no dicionário SCHEMAS).
    """
    
    try:
        df.columns = df.columns.str.upper()
        df.sort_values('DATA',ascending=False,inplace=True)
        path = os.path.join(PATHS["GOLD_LAYER"],f"{nome_indicador}.csv")

        df.to_csv(path, index=False)
        logger.success(f"Indicador '{nome_indicador.upper()}' salvo com sucesso: {path}")
        
    except Exception as e:
        logger.error(f"Erro ao salvar indicador '{nome_indicador.upper()}' na camada Gold: {e}")



In [ ]:
pd.read_parquet()

In [31]:
salvar_dados_indicadores_gold(indicador,"cny_brl")

2025-07-27 02:31:14.529 | SUCCESS  | __main__:salvar_dados_indicadores_gold:17 - Indicador 'CNY_BRL' salvo com sucesso: C:\Users\BROliveiWa12\OneDrive - NESTLE\Documents\Estudos\Engenharia de Dados\economic-indicators-pipeline\data/gold\cny_brl.xlsx


In [35]:
import os
import inspect
from pandera import DataFrameSchema
import schemas  # seu módulo com os schemas definidos


def exportar_schemas_para_yaml(output_dir: str) -> None:
    """
    Exporta todos os schemas Pandera definidos no módulo `schemas.py` para arquivos `.yaml`.

    Args:
        output_dir (str): Caminho da pasta onde os arquivos YAML serão salvos.
    """
    os.makedirs(output_dir, exist_ok=True)

    for nome, objeto in inspect.getmembers(schemas):
        if isinstance(objeto, DataFrameSchema):
            caminho_yaml = os.path.join(output_dir, f"{nome}.yaml")

            with open(caminho_yaml, "w") as f:
                f.write(objeto.to_yaml())

            print(f"✅ Schema '{nome}' exportado para {caminho_yaml}")


ModuleNotFoundError: No module named 'schemas'

NameError: name 'schemas' is not defined

In [27]:
with open(path_cny, "w") as f:
    f.write(schema_cny_brl.to_yaml())

In [29]:
pa.DataFrameSchema.from_yaml(path_cny).validate(df)

SchemaError: Column 'data' failed element-wise validator number 0: in_range(2015-07-26 00:00:00, 2025-07-26 00:00:00) failure cases: 2015-07-24 00:00:00

In [ ]:
schema_cny_brl.validate(S)

SchemaError: Column 'data' failed element-wise validator number 0: in_range(2015-07-26 00:00:00, 2025-07-26 00:00:00) failure cases: 2015-07-24 00:00:00

In [ ]:
import yaml
import black

In [8]:
import black
import yaml


ModuleNotFoundError: No module named 'black'

In [5]:
import pandera.io as io

ImportError: IO and formatting requires 'pyyaml', 'black' and 'frictionless'to be installed.
You can install pandera together with the IO dependencies with:
pip install pandera[io]


In [1]:
    import pandera as pa
    from pandera import io

ImportError: IO and formatting requires 'pyyaml', 'black' and 'frictionless'to be installed.
You can install pandera together with the IO dependencies with:
pip install pandera[io]


In [6]:
schema_cny_brl

NameError: name 'schema_cny_brl' is not defined

In [6]:
schema_cny_brl.to_yaml(os.path.join(PATHS["METRICS"], "schemas", "cny_brl.yaml"))

ImportError: IO and formatting requires 'pyyaml', 'black' and 'frictionless'to be installed.
You can install pandera together with the IO dependencies with:
pip install pandera[io]


In [2]:
pip install 'pandera[io]

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'pandera[io]": Expected package name at the start of dependency specifier
    'pandera[io]
    ^


In [ ]:
schema_cny_brl.to

In [ ]:
inicial = dt.datetime.strptime(DATA_INICIAL, '%d/%m/%Y')
final = dt.datetime.strptime(DATA_FINAL, '%d/%m/%Y')

In [ ]:
schema_ars_brl = pa.DataFrameSchema(
    name="ARS_BRL",
    description="Tabela contendo a série histórica da taxa de câmbio diária entre o Peso Argentino (ARS) e o Real Brasileiro (BRL), disponibilizada pelo Banco Central do Brasil (BACEN).",
    columns={
        "data": pa.Column(
            pa.DateTime,
            nullable=False,
            unique=True,
            checks=[pa.Check.in_range(inicial, final)],
            description="Data da cotação da taxa de câmbio ARS/BRL. Formato YYYY-MM-DD.",
        ),
        "ars_brl": pa.Column(
            pa.Float,
            nullable=False,
            checks=pa.Check.greater_than(0),
            description="Valor da taxa de câmbio do Peso Argentino (ARS) para o Real (BRL).",
        ),
    },
)

In [ ]:
schema_ars_brl.validate(pd.read_parquet(path))